# Steps for Data Preparation

**Data Loading and Initial Inspection:** The notebook starts by loading the loan data from a CSV file into a NumPy array.

It then checks for the presence of missing values (NaNs) and calculates some basic statistics (min, mean, max) for the numerical columns.


**Separating Data Types:** The columns are separated into those containing numerical data and those containing string data based on the presence of NaNs in the initial mean calculation.


**Re-importing Data by Type:** The data is re-imported from the CSV, this time specifying the data type (str for string columns and float for numerical columns with a filling value for missing data).

**Header Separation:** The header row is loaded separately and split into headers for the string and numerical data based on the column separation done earlier.

**Checkpoint:** A function is defined and used to create checkpoints, saving the current state of the headers and data arrays.

**Manipulating String Columns:**
The 'issue_d' column is cleaned by removing '-15' and then converted to numerical month representations (0-12).

The 'loan_status' column is categorized into 'good' (1) and 'bad' (0) based on a predefined list of bad statuses.

The 'term' column is cleaned by removing ' months' and any empty strings are replaced with '60', then the column is converted to integers.

The 'sub_grade' column is filled for missing values based on the 'grade' and a new category 'H1' is introduced for remaining missing values. The original 'grade' column is then removed as 'sub_grade' provides a more granular detail.

The 'sub_grade' values are converted into numerical representations using a dictionary mapping.

The 'verification_status' column is categorized into 'verified' (1) and 'not verified' or empty strings (0).

The 'url' column is cleaned to extract the loan ID, and then removed as it is redundant with the 'id' column in the numerical data.

The 'addr_state' column is categorized into geographical regions and empty strings are assigned to category '0'.

Converting String Data to Numeric: The entire string data array is converted to the integer data type.

Checkpoint (Strings): A checkpoint is created for the processed string data.

**Manipulating Numeric Columns:**
The filling values introduced during the initial import of numerical data are replaced with the minimum or maximum values of their respective columns, depending on the column.


**Currency Conversion:**
The exchange rates between EUR and USD are loaded from a separate CSV file.

Exchange rates are assigned to each row based on the 'issue_date' (month). Missing issue dates are assigned the mean exchange rate.

The exchange rates are added as a new column to the numerical data.

New columns are created for loan amounts, funded amounts, installments, and total payments converted from USD to EUR.

The headers are updated to reflect the new currency columns.

The numerical data is reordered to group the USD and EUR currency columns together.

**Interest Rate Adjustment:** The 'int_rate' column is divided by 100 to represent the interest rate as a decimal.

**Checkpoint (Numeric):** A checkpoint is created for the processed numerical data.

**Creating a Complete Dataset:** The processed numerical and string data arrays are combined horizontally.

**Sorting Data:** The combined dataset is sorted based on the 'id' column.

**Final Data Preparation:** The header row is added back to the top of the combined dataset.

**Saving Processed Data:** The final processed dataset is saved to a new CSV file named "loan-data-processed.csv".

In [1]:
import numpy as np

//for easy view - improve the way we see the data: stop numpy from numpy using scientific notation and number of words increases

In [2]:
np.set_printoptions(suppress=True, linewidth=100, precision=2)

In [6]:
raw_data_np = np.genfromtxt("/content/loan-data.csv", delimiter=';', skip_header=1, autostrip=True, encoding='latin1')
raw_data_np

array([[48010226.  ,         nan,    35000.  , ...,         nan,         nan,     9452.96],
       [57693261.  ,         nan,    30000.  , ...,         nan,         nan,     4679.7 ],
       [59432726.  ,         nan,    15000.  , ...,         nan,         nan,     1969.83],
       ...,
       [50415990.  ,         nan,    10000.  , ...,         nan,         nan,     2185.64],
       [46154151.  ,         nan,         nan, ...,         nan,         nan,     3199.4 ],
       [66055249.  ,         nan,    10000.  , ...,         nan,         nan,      301.9 ]])

Checking for incomplete Data

In [7]:
np.isnan(raw_data_np).sum()

np.int64(88005)

In [8]:
temp_fill = np.nanmax(raw_data_np) + 1
temp_mean = np.nanmean(raw_data_np, axis = 0)

/tmp/ipython-input-1910991242.py:2: RuntimeWarning: Mean of empty slice
  temp_mean = np.nanmean(raw_data_np, axis = 0)


/tmp/ipython-input-1910991242.py:2: RuntimeWarning: Mean of empty slice
  temp_mean = np.nanmean(raw_data_np, axis = 0) = we may have entire column with nan

In [9]:
temp_mean

array([54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
            440.92,         nan,         nan,         nan,         nan,         nan,     3143.85])

8 columns have nan means we might have strings or nan in that columns

extract min and max of each numerical column

In [10]:
temp_stats = np.array([np.nanmin(raw_data_np, axis = 0), temp_mean, np.nanmax(raw_data_np, axis = 0)])

/tmp/ipython-input-1993055947.py:1: RuntimeWarning: All-NaN slice encountered
  temp_stats = np.array([np.nanmin(raw_data_np, axis = 0), temp_mean, np.nanmax(raw_data_np, axis = 0)])


In [11]:
temp_stats

array([[  373332.  ,         nan,     1000.  ,         nan,     1000.  ,         nan,        6.  ,
              31.42,         nan,         nan,         nan,         nan,         nan,        0.  ],
       [54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
             440.92,         nan,         nan,         nan,         nan,         nan,     3143.85],
       [68616519.  ,         nan,    35000.  ,         nan,    35000.  ,         nan,       28.99,
            1372.97,         nan,         nan,         nan,         nan,         nan,    41913.62]])

Sliptting up data set into numerical and String data

In [13]:
column_strings = np.argwhere(np.isnan(temp_mean)).squeeze()
column_strings

array([ 1,  3,  5,  8,  9, 10, 11, 12])

In [15]:
column_numeric = np.argwhere(np.isnan(temp_mean) == False).squeeze()
column_numeric

array([ 0,  2,  4,  6,  7, 13])

Reimporting the dataset

In [16]:
loan_data_string = np.genfromtxt("/content/loan-data.csv", delimiter=';', skip_header=1, autostrip=True, encoding='latin1', usecols=column_strings, dtype=str)
loan_data_string

array([['May-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

In [17]:
loan_data_numeric = np.genfromtxt("/content/loan-data.csv", delimiter=';', skip_header=1, autostrip=True, encoding='latin1', usecols=column_numeric, filling_values=temp_fill)
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

The Names of the columns

In [19]:
headers = np.genfromtxt("/content/loan-data.csv", delimiter=';', autostrip=True, encoding='latin1', skip_footer=raw_data_np.shape[0], dtype=str)
headers

array(['id', 'issue_d', 'loan_amnt', 'loan_status', 'funded_amnt', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state',
       'total_pymnt'], dtype='<U19')

In [20]:
header_strings, header_numeric = headers[column_strings], headers[column_numeric]

In [21]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

In [22]:
header_strings

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

Creating checkpoints : places throughout the code whee we store a copy of our dataset. we want to avoid losing data

In [23]:
def checkpoint(file_name, checkpoint_header, checkpoint_data):
  np.savez(file_name, header = checkpoint_header, data = checkpoint_data)
  checkpoint_variable = np.load(file_name + ".npz")
  return(checkpoint_variable)

In [24]:
checkpoint_test = checkpoint("checkpoint-test", header_strings, loan_data_string)

In [25]:
checkpoint_test

NpzFile 'checkpoint-test.npz' with keys: header, data

In [26]:
checkpoint_test['header']

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [27]:
checkpoint_test['data']

array([['May-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

In [28]:
np.array_equal(checkpoint_test['data'], loan_data_string)

True

Manipulating String Columns

In [29]:
header_strings

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [30]:
header_strings[0] = "issue_date"

In [31]:
header_strings

array(['issue_date', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [32]:
loan_data_string

array([['May-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

Issue Date

In [34]:
np.unique(loan_data_string[:, 0])

array(['', 'Apr-15', 'Aug-15', 'Dec-15', 'Feb-15', 'Jan-15', 'Jul-15', 'Jun-15', 'Mar-15',
       'May-15', 'Nov-15', 'Oct-15', 'Sep-15'], dtype='<U69')

In [35]:
# striping the issue date becuase the year is 2015
loan_data_string[:, 0]=np.chararray.strip(loan_data_string[:,0], "-15")

/tmp/ipython-input-3468262217.py:2: DeprecationWarning: `np.chararray` is deprecated and will be removed from the main namespace in the future. Use an array with a string or bytes dtype instead.
  loan_data_string[:, 0]=np.chararray.strip(loan_data_string[:,0], "-15")


In [36]:
np.unique(loan_data_string[:, 0])

array(['', 'Apr', 'Aug', 'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov', 'Oct', 'Sep'],
      dtype='<U69')

In [57]:
months = np.array(['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
for i in range(13):
  loan_data_string[:,0] = np.where(loan_data_string[:,0] == months[i], str(i), loan_data_string[:,0])

loan_data_string[:,0] = loan_data_string[:,0].astype(int)

In [44]:
np.unique(loan_data_string[:,0])

array(['0', '1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='<U69')

Loan Status

In [45]:
header_strings

array(['issue_date', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [46]:
loan_data_string[:,1]

array(['Current', 'Current', 'Current', ..., 'Current', 'Current', 'Current'], dtype='<U69')

In [47]:
np.unique(loan_data_string[:,1])

array(['', 'Charged Off', 'Current', 'Default', 'Fully Paid', 'In Grace Period', 'Issued',
       'Late (16-30 days)', 'Late (31-120 days)'], dtype='<U69')

In [48]:
np.unique(loan_data_string[:,1]).size

9

We can use np.where() to assign a different number to each one.
We need to slpit all possible values into either gropup("good" or "bad")

In [49]:
status_bad = np.array(['', 'Charged Off', 'Default', 'Late (31-120 days)'])

In [50]:
loan_data_string[:, 1] = np.where(np.isin(loan_data_string[:,1], status_bad), 0, 1)

In [51]:
loan_data_string[:,1]

array(['1', '1', '1', ..., '1', '1', '1'], dtype='<U69')

Term

In [52]:
header_strings

array(['issue_date', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [53]:
np.unique(loan_data_string[:,2])

array(['', '36 months', '60 months'], dtype='<U69')

In [54]:
loan_data_string[:,2] = np.chararray.strip(loan_data_string[:,2], " months")
loan_data_string[:,2]

/tmp/ipython-input-705060273.py:1: DeprecationWarning: `np.chararray` is deprecated and will be removed from the main namespace in the future. Use an array with a string or bytes dtype instead.
  loan_data_string[:,2] = np.chararray.strip(loan_data_string[:,2], " months")


array(['36', '36', '36', ..., '36', '36', '36'], dtype='<U69')

In [55]:
header_strings[2] = "term_months"

In [56]:
header_strings

array(['issue_date', 'loan_status', 'term_months', 'grade', 'sub_grade', 'verification_status',
       'url', 'addr_state'], dtype='<U19')

In [59]:
loan_data_string[:,2] = np.where(loan_data_string[:,2] == '', str(60), loan_data_string[:,2])
loan_data_string[:,2]

array(['36', '36', '36', ..., '36', '36', '36'], dtype='<U69')

In [60]:
np.unique(loan_data_string[:,2])

array(['36', '60'], dtype='<U69')

Grade and Subgrade

In [61]:
header_strings

array(['issue_date', 'loan_status', 'term_months', 'grade', 'sub_grade', 'verification_status',
       'url', 'addr_state'], dtype='<U19')

In [62]:
np.unique(loan_data_string[:,3])

array(['', 'A', 'B', 'C', 'D', 'E', 'F', 'G'], dtype='<U69')

In [63]:
loan_data_string[:,4]

array(['C3', 'A5', 'B5', ..., 'A5', 'D2', 'A4'], dtype='<U69')

In [64]:
np.unique(loan_data_string[:,4])

array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
       'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
       'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69')

Filling Sub Grade

In [65]:
for i in np.unique(loan_data_string[:,3])[1:]:
  loan_data_string[:,4] = np.where((loan_data_string[:,4] == '') & (loan_data_string[:,3] == i), i + '5', loan_data_string[:,4])

In [66]:
np.unique(loan_data_string[:, 4], return_counts=True)

(array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
        'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
        'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69'),
 array([  9, 285, 278, 239, 323, 592, 509, 517, 530, 553, 633, 629, 567, 586, 564, 577, 391, 267,
        250, 255, 288, 235, 162, 171, 139, 160,  94,  52,  34,  43,  24,  19,  10,   3,   7,   5]))

//'' = 9 which is less about 0.5% so for missing elements we use a new grading system H1

In [67]:
loan_data_string[:,4] = np.where((loan_data_string[:,4] == '') , 'H1', loan_data_string[:,4])

In [68]:
np.unique(loan_data_string[:,4])

array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5',
       'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5',
       'G1', 'G2', 'G3', 'G4', 'G5', 'H1'], dtype='<U69')

Remove grade since details of grade are in subgrade

In [70]:
loan_data_string = np.delete(loan_data_string, 3, axis=1)

In [71]:
loan_data_string[:,3]

array(['C3', 'A5', 'B5', ..., 'A5', 'D2', 'A4'], dtype='<U69')

In [72]:
header_strings = np.delete(header_strings,3)

In [73]:
header_strings[3]

np.str_('sub_grade')

Converting Subgrade

In [74]:
np.unique(loan_data_string[:,3])

array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5',
       'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5',
       'G1', 'G2', 'G3', 'G4', 'G5', 'H1'], dtype='<U69')

In [75]:
keys = list(np.unique(loan_data_string[:,3]))
values = list(range(1, np.unique(loan_data_string[:,3]).shape[0] + 1))
dict_sub_grade = dict(zip(keys, values))

In [76]:
dict_sub_grade

{np.str_('A1'): 1,
 np.str_('A2'): 2,
 np.str_('A3'): 3,
 np.str_('A4'): 4,
 np.str_('A5'): 5,
 np.str_('B1'): 6,
 np.str_('B2'): 7,
 np.str_('B3'): 8,
 np.str_('B4'): 9,
 np.str_('B5'): 10,
 np.str_('C1'): 11,
 np.str_('C2'): 12,
 np.str_('C3'): 13,
 np.str_('C4'): 14,
 np.str_('C5'): 15,
 np.str_('D1'): 16,
 np.str_('D2'): 17,
 np.str_('D3'): 18,
 np.str_('D4'): 19,
 np.str_('D5'): 20,
 np.str_('E1'): 21,
 np.str_('E2'): 22,
 np.str_('E3'): 23,
 np.str_('E4'): 24,
 np.str_('E5'): 25,
 np.str_('F1'): 26,
 np.str_('F2'): 27,
 np.str_('F3'): 28,
 np.str_('F4'): 29,
 np.str_('F5'): 30,
 np.str_('G1'): 31,
 np.str_('G2'): 32,
 np.str_('G3'): 33,
 np.str_('G4'): 34,
 np.str_('G5'): 35,
 np.str_('H1'): 36}

In [79]:
for i in np.unique(loan_data_string[:,3]):
  loan_data_string[:,3] = np.where(loan_data_string[:,3] == i, str(dict_sub_grade[i]), loan_data_string[:,3])

In [80]:
np.unique(loan_data_string[:,3])

array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36',
       '4', '5', '6', '7', '8', '9'], dtype='<U69')

Verification Status

In [81]:
header_strings

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [82]:
np.unique(loan_data_string[:,4])

array(['', 'Not Verified', 'Source Verified', 'Verified'], dtype='<U69')

Not verified and '' = 0 because theya re bad

In [84]:
loan_data_string[:,4] = np.where((loan_data_string[:,4] == '') | (loan_data_string[:,4] == 'Not Verified'), 0, 1)

In [85]:
np.unique(loan_data_string[:,4])

array(['0', '1'], dtype='<U69')

URL

In [86]:
loan_data_string[:,5]

array(['https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', ...,
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249'], dtype='<U69')

In [87]:
np.chararray.strip(loan_data_string[:,5], "https://www.lendingclub.com/browse/loanDetail.action?loan_id=")

/tmp/ipython-input-2953293532.py:1: DeprecationWarning: `np.chararray` is deprecated and will be removed from the main namespace in the future. Use an array with a string or bytes dtype instead.
  np.chararray.strip(loan_data_string[:,5], "https://www.lendingclub.com/browse/loanDetail.action?loan_id=")


array(['48010226', '57693261', '59432726', ..., '50415990', '46154151', '66055249'], dtype='<U69')

In [88]:
loan_data_string[:,5] = np.chararray.strip(loan_data_string[:,5], "https://www.lendingclub.com/browse/loanDetail.action?loan_id=")

/tmp/ipython-input-4252045116.py:1: DeprecationWarning: `np.chararray` is deprecated and will be removed from the main namespace in the future. Use an array with a string or bytes dtype instead.
  loan_data_string[:,5] = np.chararray.strip(loan_data_string[:,5], "https://www.lendingclub.com/browse/loanDetail.action?loan_id=")


In [89]:
headers

array(['id', 'issue_d', 'loan_amnt', 'loan_status', 'funded_amnt', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state',
       'total_pymnt'], dtype='<U19')

In [90]:
loan_data_numeric[:,0].astype(dtype=np.int32)

array([48010226, 57693261, 59432726, ..., 50415990, 46154151, 66055249], dtype=int32)

In [91]:
loan_data_string[:,5].astype(dtype=np.int32)

array([48010226, 57693261, 59432726, ..., 50415990, 46154151, 66055249], dtype=int32)

both url and installment are same so remove url

In [92]:
np.array_equal(loan_data_string[:,5].astype(dtype=np.int32), loan_data_numeric[:,0].astype(dtype=np.int32))

True

In [93]:
loan_data_string = np.delete(loan_data_string, 5, axis=1)
header_strings = np.delete(header_strings, 5)

In [94]:
loan_data_string[:,5]

array(['CA', 'NY', 'PA', ..., 'CA', 'OH', 'IL'], dtype='<U69')

In [95]:
header_strings

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'addr_state'], dtype='<U19')

In [96]:
loan_data_numeric[:,0]

array([48010226., 57693261., 59432726., ..., 50415990., 46154151., 66055249.])

In [97]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

State Address

In [98]:
header_strings

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'addr_state'], dtype='<U19')

In [99]:
header_strings[5] = "state_address"

In [102]:
np.unique(loan_data_string[:,5])

array(['', 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IL', 'IN',
       'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH',
       'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA',
       'VT', 'WA', 'WI', 'WV', 'WY'], dtype='<U69')

In [103]:
np.unique(loan_data_string[:,5]).size

50

In [104]:
np.unique(loan_data_string[:,5], return_counts=True)

(array(['', 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IL', 'IN',
        'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH',
        'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA',
        'VT', 'WA', 'WI', 'WV', 'WY'], dtype='<U69'),
 array([ 500,   26,  119,   74,  220, 1336,  201,  143,   27,   27,  690,  321,   44,  389,  152,
          84,   84,  116,  210,  222,   10,  267,  156,  160,   61,   28,  261,   16,   25,   58,
         341,   57,  130,  777,  312,   83,  108,  320,   40,  107,   24,  143,  758,   74,  242,
          17,  216,  148,   49,   27]))

In [105]:
from os import stat
state_name, state_counts = np.unique(loan_data_string[:,5], return_counts=True)
state_count_sorted = np.argsort(-state_counts)
state_name[state_count_sorted], state_counts[state_count_sorted]

(array(['CA', 'NY', 'TX', 'FL', '', 'IL', 'NJ', 'GA', 'PA', 'OH', 'MI', 'NC', 'VA', 'MD', 'AZ',
        'WA', 'MA', 'CO', 'MO', 'MN', 'IN', 'WI', 'TN', 'CT', 'NV', 'AL', 'LA', 'OR', 'SC', 'KY',
        'KS', 'OK', 'UT', 'AR', 'MS', 'NH', 'NM', 'WV', 'HI', 'RI', 'MT', 'DC', 'DE', 'WY', 'AK',
        'NE', 'SD', 'VT', 'ND', 'ME'], dtype='<U69'),
 array([1336,  777,  758,  690,  500,  389,  341,  321,  320,  312,  267,  261,  242,  222,  220,
         216,  210,  201,  160,  156,  152,  148,  143,  143,  130,  119,  116,  108,  107,   84,
          84,   83,   74,   74,   61,   58,   57,   49,   44,   40,   28,   27,   27,   27,   26,
          25,   24,   17,   16,   10]))

In [107]:
loan_data_string[:,5] = np.where(loan_data_string[:,5] == '', str(0), loan_data_string[:,5])

In [108]:
states_west = np.array(['WA', 'OR', 'CA', 'NV', 'ID', 'MT', 'WY', 'UT', 'CO', 'AZ', 'NM', "HI", "AK"])
states_south = np.array(['TX', 'OK', 'AR', 'LA', 'MS', 'AL', 'TN', 'KY', 'FL', 'GA', 'SC', 'NC', 'VA', 'WW", "MD', 'DE', 'DC'])
states_midwest= np.array(['ND', 'SD', 'NE', 'KS', 'MN', 'IA', 'MO', 'WI', 'IL', 'IN', 'MI', 'OH'])
states_east = np.array(['PA', 'NY', 'NJ', 'CT', 'MA', 'VT', 'NH', 'MF', 'RI'])

In [116]:
loan_data_string[:,5] = np.where(np.isin(loan_data_string[:,5], states_west), '1', loan_data_string[:,5])
loan_data_string[:,5] = np.where(np.isin(loan_data_string[:,5], states_south), '2', loan_data_string[:,5])
loan_data_string[:,5] = np.where(np.isin(loan_data_string[:,5], states_midwest), '3', loan_data_string[:,5])
loan_data_string[:,5] = np.where(np.isin(loan_data_string[:,5], states_east), '4', loan_data_string[:,5])
loan_data_string[:,5] = np.where((loan_data_string[:,5] != '1') & (loan_data_string[:,5] != '2') & (loan_data_string[:,5] != '3') & (loan_data_string[:,5] != '4') & (loan_data_string[:,5] != '0'), '0', loan_data_string[:,5])

In [117]:
np.unique(loan_data_string[:,5])

array(['0', '1', '2', '3', '4'], dtype='<U69')

Converting strings to numbers

In [118]:
loan_data_string

array([['5', '1', '36', '13', '1', '1'],
       ['0', '1', '36', '5', '1', '4'],
       ['9', '1', '36', '10', '1', '4'],
       ...,
       ['6', '1', '36', '5', '1', '1'],
       ['4', '1', '36', '17', '1', '3'],
       ['12', '1', '36', '4', '0', '3']], dtype='<U69')

In [126]:
loan_data_string=loan_data_string.astype(np.int32)

Checkpoint1 : Strings

In [127]:
checkpoint_string_test = checkpoint("Cheskpoint-Strings", header_strings, loan_data_string)

In [128]:
checkpoint_string_test['header']

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'state_address'], dtype='<U19')

In [129]:
checkpoint_string_test['data']

array([[ 5,  1, 36, 13,  1,  1],
       [ 0,  1, 36,  5,  1,  4],
       [ 9,  1, 36, 10,  1,  4],
       ...,
       [ 6,  1, 36,  5,  1,  1],
       [ 4,  1, 36, 17,  1,  3],
       [12,  1, 36,  4,  0,  3]], dtype=int32)

In [130]:
np.array_equal(checkpoint_string_test['data'], loan_data_string)

True

Manipulating Numeric Columns

In [131]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

In [132]:
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

In [133]:
np.isnan(loan_data_numeric).sum()

np.int64(0)

Substitute Filling values becuase we filled with filler values so we make them worst values

ID

In [134]:
temp_fill

np.float64(68616520.0)

In [135]:
np.isin(loan_data_numeric[:,0], temp_fill)

array([False, False, False, ..., False, False, False])

In [136]:
np.isin(loan_data_numeric[:,0], temp_fill).sum()
# no filling values used if answer = 0

np.int64(0)

TemporaryStats

In [138]:
temp_stats[:, column_numeric]
# temp min, mean, max is shown one belonging to each row

array([[  373332.  ,     1000.  ,     1000.  ,        6.  ,       31.42,        0.  ],
       [54015809.19,    15273.46,    15311.04,       16.62,      440.92,     3143.85],
       [68616519.  ,    35000.  ,    35000.  ,       28.99,     1372.97,    41913.62]])

funded value we are keeping min value

In [139]:
loan_data_numeric[:,2]

array([35000., 30000., 15000., ..., 10000., 10000., 10000.])

In [140]:
loan_data_numeric[:,2] = np.where(loan_data_numeric[:,2] == temp_fill, temp_stats[0, column_numeric[2]], loan_data_numeric[:,2])

In [141]:
loan_data_numeric[:,2]

array([35000., 30000., 15000., ..., 10000., 10000., 10000.])

In [142]:
temp_stats[0, column_numeric[3]]

np.float64(6.0)

Loan Amount, Interest Rate, Total Payment, Installment we are keeping max value

In [143]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

In [144]:
for i in [1,3,4,5]:
  loan_data_numeric[:,i] = np.where(loan_data_numeric[:,i] == temp_fill, temp_stats[2, column_numeric[i]], loan_data_numeric[:,i])

In [145]:
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  ,       28.99,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  ,       28.99,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  ,       28.99,     1372.97,     2185.64],
       [46154151.  ,    35000.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  ,       28.99,      309.97,      301.9 ]])

Currency change

In [ ]:
# find the exchange rate avaliabe in the  cv of eur_usd.csv

In [147]:
EUR_USD = np.genfromtxt("/content/EUR-USD.csv", delimiter = ',', autostrip=True,dtype = str)
EUR_USD

array([['Open', 'High', 'Low', 'Close', 'Volume'],
       ['1.2098628282546997', '1.2098628282546997', '1.11055588722229', '1.1287955045700073', '0'],
       ['1.1287955045700073', '1.1484194993972778', '1.117680549621582', '1.1205360889434814',
        '0'],
       ['1.119795799255371', '1.1240400075912476', '1.0460032224655151', '1.0830246210098267',
        '0'],
       ['1.0741022825241089', '1.1247594356536865', '1.0521597862243652', '1.1114321947097778',
        '0'],
       ['1.1215037107467651', '1.145304799079895', '1.0821995735168457', '1.0960345268249512',
        '0'],
       ['1.095902442932129', '1.1428401470184326', '1.0888904333114624', '1.122296690940857', '0'],
       ['1.1134989261627197', '1.1219995021820068', '1.081270456314087', '1.0939244031906128',
        '0'],
       ['1.0969001054763794', '1.1705996990203857', '1.0850305557250977', '1.1340054273605347',
        '0'],
       ['1.1225990056991577', '1.1460003852844238', '1.1089695692062378', '1.1255937814712524

In [ ]:
# create a column to store exchange rates

In [148]:
loan_data_string[:,0]

array([ 5,  0,  9, ...,  6,  4, 12], dtype=int32)

In [152]:
exchange_rates = loan_data_string[:,0].astype(float)
for i in range(1,13):
  exchange_rates = np.where(exchange_rates == i, EUR_USD[i][1], exchange_rates)

exchange_rates = np.where(exchange_rates == 0, np.mean(EUR_USD[1:,1].astype(float)), exchange_rates)

In [153]:
exchange_rates.shape

(10000,)

In [154]:
loan_data_numeric.shape

(10000, 6)

In [155]:
exchange_rates = np.reshape(exchange_rates, (10000,1))

In [156]:
loan_data_numeric = np.hstack((loan_data_numeric, exchange_rates))

In [157]:
header_numeric = np.concatenate((header_numeric, np.array(['exchange_rates'])))
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt',
       'exchange_rates'], dtype='<U19')

From USD to EUR

In [158]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt',
       'exchange_rates'], dtype='<U19')

In [159]:
columns_dollars = np.array([1,2,4,5])

In [160]:
loan_data_numeric[:, [columns_dollars]]

array([[['35000.0', '35000.0', '1184.86', '9452.96']],

       [['30000.0', '30000.0', '938.57', '4679.7']],

       [['15000.0', '15000.0', '494.86', '1969.83']],

       ...,

       [['10000.0', '10000.0', '1372.97', '2185.64']],

       [['35000.0', '10000.0', '354.3', '3199.4']],

       [['10000.0', '10000.0', '309.97', '301.9']]], dtype='<U32')

In [162]:
loan_data_numeric = loan_data_numeric.astype(float)
for i in columns_dollars:
  loan_data_numeric = np.hstack((loan_data_numeric, np.reshape(loan_data_numeric[:,i] / loan_data_numeric[:,6], (10000,1))))

/tmp/ipython-input-3898716547.py:3: RuntimeWarning: divide by zero encountered in divide
  loan_data_numeric = np.hstack((loan_data_numeric, np.reshape(loan_data_numeric[:,i] / loan_data_numeric[:,6], (10000,1))))
/tmp/ipython-input-3898716547.py:3: RuntimeWarning: invalid value encountered in divide
  loan_data_numeric = np.hstack((loan_data_numeric, np.reshape(loan_data_numeric[:,i] / loan_data_numeric[:,6], (10000,1))))


In [164]:
loan_data_numeric.shape

(10000, 11)

In [165]:
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  , ...,     7000.  ,      236.97,     1890.59],
       [57693261.  ,    30000.  ,    30000.  , ...,         inf,         inf,         inf],
       [59432726.  ,    15000.  ,    15000.  , ...,     1666.67,       54.98,      218.87],
       ...,
       [50415990.  ,    10000.  ,    10000.  , ...,     1666.67,      228.83,      364.27],
       [46154151.  ,    35000.  ,    10000.  , ...,     2500.  ,       88.58,      799.85],
       [66055249.  ,    10000.  ,    10000.  , ...,      833.33,       25.83,       25.16]])

Expand header

In [166]:
header_additional = np.array([column_name + '_EUR' for column_name in header_numeric[columns_dollars]])

In [167]:
header_additional

array(['loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'], dtype='<U15')

In [169]:
header_numeric = np.concatenate((header_numeric, header_additional))

In [170]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt',
       'exchange_rates', 'loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'],
      dtype='<U19')

In [171]:
header_numeric[columns_dollars] = np.array([column_name + '_USD' for column_name in header_numeric[columns_dollars]])

In [172]:
header_numeric

array(['id', 'loan_amnt_USD', 'funded_amnt_USD', 'int_rate', 'installment_USD', 'total_pymnt_USD',
       'exchange_rates', 'loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'],
      dtype='<U19')

In [173]:
column_order = [0,1,7,2,8,3,4,9,5,10,6]

In [174]:
header_numeric[column_order]

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
       'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR',
       'exchange_rates'], dtype='<U19')

In [175]:
header_numeric = header_numeric[column_order]

In [176]:
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  , ...,     7000.  ,      236.97,     1890.59],
       [57693261.  ,    30000.  ,    30000.  , ...,         inf,         inf,         inf],
       [59432726.  ,    15000.  ,    15000.  , ...,     1666.67,       54.98,      218.87],
       ...,
       [50415990.  ,    10000.  ,    10000.  , ...,     1666.67,      228.83,      364.27],
       [46154151.  ,    35000.  ,    10000.  , ...,     2500.  ,       88.58,      799.85],
       [66055249.  ,    10000.  ,    10000.  , ...,      833.33,       25.83,       25.16]])

In [177]:
loan_data_numeric = loan_data_numeric[:,column_order]

Interest rate

In [178]:
header_numeric

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
       'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR',
       'exchange_rates'], dtype='<U19')

In [179]:
loan_data_numeric[:,5]
# stored as percentage

array([13.33, 28.99, 28.99, ..., 28.99, 16.55, 28.99])

In [180]:
loan_data_numeric[:,5] = loan_data_numeric[:,5]/100
# divide by 100

In [181]:
loan_data_numeric[:,5]

array([0.13, 0.29, 0.29, ..., 0.29, 0.17, 0.29])

Checkpoint 2 Numeric

In [182]:
checkpoint_numeric_test = checkpoint("Checkpoint-Numeric", header_numeric, loan_data_numeric)

In [183]:
checkpoint_numeric_test['header']

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
       'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR',
       'exchange_rates'], dtype='<U19')

In [184]:
checkpoint_numeric_test['data']

array([[48010226.  ,    35000.  ,     7000.  , ...,     9452.96,     1890.59,        5.  ],
       [57693261.  ,    30000.  ,         inf, ...,     4679.7 ,         inf,        0.  ],
       [59432726.  ,    15000.  ,     1666.67, ...,     1969.83,      218.87,        9.  ],
       ...,
       [50415990.  ,    10000.  ,     1666.67, ...,     2185.64,      364.27,        6.  ],
       [46154151.  ,    35000.  ,     8750.  , ...,     3199.4 ,      799.85,        4.  ],
       [66055249.  ,    10000.  ,      833.33, ...,      301.9 ,       25.16,       12.  ]])

Creating a complete dataset

In [186]:
# using ladder approach
checkpoint_string_test['data'].shape

(10000, 6)

In [187]:
checkpoint_numeric_test['data'].shape

(10000, 11)

In [199]:
loan_data = np.hstack((checkpoint_numeric_test['data'], checkpoint_string_test['data']))

In [200]:
loan_data

array([[48010226.  ,    35000.  ,     7000.  , ...,       13.  ,        1.  ,        1.  ],
       [57693261.  ,    30000.  ,         inf, ...,        5.  ,        1.  ,        4.  ],
       [59432726.  ,    15000.  ,     1666.67, ...,       10.  ,        1.  ,        4.  ],
       ...,
       [50415990.  ,    10000.  ,     1666.67, ...,        5.  ,        1.  ,        1.  ],
       [46154151.  ,    35000.  ,     8750.  , ...,       17.  ,        1.  ,        3.  ],
       [66055249.  ,    10000.  ,      833.33, ...,        4.  ,        0.  ,        3.  ]])

In [201]:
np.isnan(loan_data).sum()

np.int64(26)

In [202]:
header = np.concatenate((checkpoint_numeric_test['header'], checkpoint_string_test['header']))

Sorting based on index

In [203]:
loan_data = loan_data[np.argsort(loan_data[:,0])]

In [204]:
loan_data

array([[  373332.  ,     9950.  ,      995.  , ...,       21.  ,        0.  ,        1.  ],
       [  575239.  ,    12000.  ,     1200.  , ...,       25.  ,        1.  ,        0.  ],
       [  707689.  ,    10000.  ,     5000.  , ...,       13.  ,        1.  ,        0.  ],
       ...,
       [68614880.  ,     5600.  ,      466.67, ...,        8.  ,        1.  ,        1.  ],
       [68615915.  ,     4000.  ,      333.33, ...,       10.  ,        1.  ,        2.  ],
       [68616519.  ,    21600.  ,     1800.  , ...,        3.  ,        0.  ,        2.  ]])

In [205]:
np.argsort(loan_data[:,0])

array([   0,    1,    2, ..., 9997, 9998, 9999])

In [ ]:
# Storing the entire dataset

In [206]:
loan_data = np.vstack((header, loan_data))

In [207]:
np.savetxt("loan-data-processed.csv", loan_data, fmt='%s', delimiter=',')